In [ ]:
#百度语音识别

from aip import AipSpeech

 

""" 你的 APPID AK SK """

APP_ID = ''

API_KEY = ''

SECRET_KEY = ''

 

client = AipSpeech(APP_ID, API_KEY, SECRET_KEY)

 

# 读取文件

def get_file_content(filePath):

    with open(filePath, 'rb') as fp:

        return fp.read()

 

# 识别本地文件

rtn = client.asr(get_file_content('16k-23850.amr'), 'amr', 16000, {

    'dev_pid': 1536,

})

 

print(rtn['result'][0])


In [ ]:
#阿里语音识别

# -*- coding: utf-8 -*-
import os
import time
import threading
import ali_speech
from ali_speech.callbacks import SpeechTranscriberCallback
from ali_speech.constant import ASRFormat
from ali_speech.constant import ASRSampleRate
class MyCallback(SpeechTranscriberCallback):
    """
    构造函数的参数没有要求，可根据需要设置添加
    示例中的name参数可作为待识别的音频文件名，用于在多线程中进行区分
    """
    def __init__(self, name='default'):
        self._name = name
    def on_started(self, message):
        print('MyCallback.OnRecognitionStarted: %s' % message)
    def on_result_changed(self, message):
        print('MyCallback.OnRecognitionResultChanged: file: %s, task_id: %s, result: %s' % (
            self._name, message['header']['task_id'], message['payload']['result']))
    def on_sentence_begin(self, message):
        print('MyCallback.on_sentence_begin: file: %s, task_id: %s, sentence_id: %s, time: %s' % (
            self._name, message['header']['task_id'], message['payload']['index'], message['payload']['time']))
    def on_sentence_end(self, message):
        print('MyCallback.on_sentence_end: file: %s, task_id: %s, sentence_id: %s, time: %s, result: %s' % (
            self._name,
            message['header']['task_id'], message['payload']['index'],
            message['payload']['time'], message['payload']['result']))
    def on_completed(self, message):
        print('MyCallback.OnRecognitionCompleted: %s' % message)
    def on_task_failed(self, message):
        print('MyCallback.OnRecognitionTaskFailed-task_id:%s, status_text:%s' % (
            message['header']['task_id'], message['header']['status_text']))
    def on_channel_closed(self):
        print('MyCallback.OnRecognitionChannelClosed')
def process(client, appkey, token):
    audio_name = 'nls-sample-16k.wav'
    callback = MyCallback(audio_name)
    transcriber = client.create_transcriber(callback)
    transcriber.set_appkey(appkey)
    transcriber.set_token(token)
    transcriber.set_format(ASRFormat.PCM)
    transcriber.set_sample_rate(ASRSampleRate.SAMPLE_RATE_16K)
    transcriber.set_enable_intermediate_result(False)
    transcriber.set_enable_punctuation_prediction(True)
    transcriber.set_enable_inverse_text_normalization(True)
    try:
        ret = transcriber.start()
        if ret < 0:
            return ret
        print('sending audio...')
        with open(audio_name, 'rb') as f:
            audio = f.read(3200)
            while audio:
                ret = transcriber.send(audio)
                if ret < 0:
                    break
                time.sleep(0.1)
                audio = f.read(3200)
        transcriber.stop()
    except Exception as e:
        print(e)
    finally:
        transcriber.close()
def process_multithread(client, appkey, token, number):
    thread_list = []
    for i in range(0, number):
        thread = threading.Thread(target=process, args=(client, appkey, token))
        thread_list.append(thread)
        thread.start()
    for thread in thread_list:
        thread.join()
if __name__ == "__main__":
    client = ali_speech.NlsClient()
    # 设置输出日志信息的级别：DEBUG、INFO、WARNING、ERROR
    client.set_log_level('INFO')
    appkey = '您的appkey'
    token = '您的Token'
    process(client, appkey, token)
    # 多线程示例
    # process_multithread(client, appkey, token, 2)

In [ ]:
#百度人脸识别api
import urllib3,base64
from urllib.parse import urlencode
import json
import cv2
import time
t1=time.time()
access_token='your token'
http=urllib3.PoolManager()
url='https://aip.baidubce.com/rest/2.0/face/v2/detect?access_token='+access_token
#2张图片
filepath='D:/作业！！！！！！！！！！！！！！！！！/大三/上/API/期末/r.jpg'
f1 = open(filepath,'rb')
frame=cv2.imread(filepath)
#参数images：图像base64编码 分别base64编码后的2张图片数据，需urlencode，半角逗号分隔，单次请求最大不超过20M
img1 = base64.b64encode(f1.read())

#这一步和官方示例代码不一样。具体为什么就不知道了。里面有json bytes str 类型关系
#这里直接拼接提示byte相关。然后就直接转str拼接了
params = {"images":str(img1,'utf-8'),"max_face_num":10}
#对base64数据进行urlencode处理
params=urlencode(params)
request=http.request('POST',
                      url,
                      body=params,
                      headers={'Content-Type':'application/x-www-form-urlencoded'})
#对返回的byte字节进行处理。Python3输出位串，而不是可读的字符串，需要进行转换

print(json.loads(request.data))  # request.data 返回有关的信息但是是json类型  然后使用json.loads 进行转换转成python支持的dict
result = json.loads(request.data)['result']  #分析返回的数据 找到有用的信息拿出来 我们这里是要拿出来人脸坐标
print(result)
face_num=json.loads(request.data)['result_num']  #获取到图片中人脸的个数

for i in range(face_num):  #使用遍历把所有的人脸都标出框

    location=result[i]['location']  #获取到人脸的坐标
    print(location)   #输出人脸坐标 left location是左上角坐标  width 宽度height高度
    cv2.rectangle(frame, (location['left'], location['top']), (location['width']+location['left'], location['height']+location['top']), (0, 0, 255), 2) #opencv的标框函数

cv2.imshow('tuxiang',frame)
cv2.waitKey(1)  #刷新界面 不然只会呈现灰色
print('运行时间是{}'.format(time.time()-t1))
time.sleep(5)  #暂停五秒  展示图片